In [1]:
import numpy as np
import tensorflow as tf
import random
from utils import *

from dynamicgem.embedding.dynAERNN  import DynAERNN 
import dgl  
import scipy as sp
import scipy.linalg as linalg
import networkx as nx
import matplotlib.pyplot as plt
from scipy.cluster.vq import kmeans,vq
from scipy import stats 
from sklearn.cluster import SpectralClustering
from sklearn import metrics

from itertools import permutations



C:\Users\conno\anaconda3\envs\dynamicgem_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\conno\anaconda3\envs\dynamicgem_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\conno\anaconda3\envs\dynamicgem_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\conno\anaconda3\envs\dynamicgem_env\l

In [2]:
!pip install pytorch

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch: started
    Running setup.py install for pytorch: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\conno\anaconda3\envs\dynamicgem_env\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\conno\\AppData\\Local\\Temp\\pip-install-utafe9io\\pytorch_3e9946fc83bc426e8dbfa805aa855765\\setup.py'"'"'; __file__='"'"'C:\\Users\\conno\\AppData\\Local\\Temp\\pip-install-utafe9io\\pytorch_3e9946fc83bc426e8dbfa805aa855765\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\conno\AppData\Local\Temp\pip-wheel-bv_g61pl'
       cwd: C:\Users\conno\AppData\Local\Temp\pip-install-utafe9io\pytorch_3e9946fc83bc426e8dbfa805aa855765\
  Complete output (5 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "C:\User

In [3]:
def read_data(id: str): #DBLP3, DBLP5, Brain, Reddit, DBLPE
    dataset_dict=dict()
    dataset_dict["DBLP3"]="Datasets/DBLP3.npz"
    dataset_dict["DBLP5"]="Datasets/DBLP5.npz"
    dataset_dict["Brain"]="Datasets/Brain.npz"
    dataset_dict["Reddit"]="Datasets/reddit.npz"
    dataset_dict["DBLPE"]="Datasets/DBLPE.npz"

    dataset = np.load(dataset_dict[id])
    adjs = dataset["adjs"] #(time, node, node)

    #Remove nodes with no connections at any timestep
    temporal_sum = np.add.reduce(adjs, axis=0, keepdims=False)
    row_sum = np.add.reduce(temporal_sum, axis=0, keepdims=False)
    non_zero_indices = np.flatnonzero(row_sum)
    adjs = adjs[:,non_zero_indices,:]
    adjs = adjs[:,:,non_zero_indices]

    #DBLPE is a dynamic featureless graph
    if id=="DBLPE":
        labels = dataset["labels"] #(nodes, time, class)

        # labels = np.argmax(labels,axis=2)
        labels=labels[non_zero_indices]
        feats=np.zeros([adjs.shape[1], adjs.shape[0], adjs.shape[2]])

        for i in range(feats.shape[1]):
            feats[:,i,:]=np.eye(feats.shape[0])
      
    #All others are static feature-full graphs
    else:
        labels = dataset["labels"] #(nodes, class)
        feats = dataset["attmats"] #(node, time, feat)

        # labels = np.argmax(labels, axis=1)
        labels = labels[non_zero_indices]
        feats = feats[non_zero_indices]

    #Other important variables
    n_nodes = adjs.shape[1]
    n_timesteps = adjs.shape[0]
    n_class = int(labels.shape[1])
    n_feat = feats.shape[2]

    #Train Val Test split
    nodes_id = list(range(n_nodes))
    random.shuffle(nodes_id)
    idx_train = nodes_id[:(7*n_nodes)//10]
    idx_train = [True if i in idx_train else False for i in list(range(n_nodes))]
    idx_val = nodes_id[(7*n_nodes)//10: (9*n_nodes)//10]
    idx_val = [True if i in idx_val else False for i in list(range(n_nodes))]
    idx_test = nodes_id[(9*n_nodes)//10: n_nodes]
    idx_test = [True if i in idx_test else False for i in list(range(n_nodes))]

    return STG_Dataset(np.array(adjs),
                        np.array(adjs),
                        np.array(feats), 
                        np.array(feats), 
                        np.array(labels), 
                        np.array(labels), 
                        n_nodes, n_timesteps, n_class, n_feat, 
                        np.array(idx_train),
                        np.array(idx_val),
                        np.array(idx_test))
    

In [4]:
def dynaernn(data):
    length=data.n_timestamps
    lookup=length-2

    dim_emb  = data.n_class
          
    tf.device('/gpu:0')

    embedding = DynAERNN(d   = dim_emb,
        beta           = 5,
        n_prev_graphs  = lookup,
        nu1            = 1e-6,
        nu2            = 1e-6,
        n_aeunits      = [50, 30],
        n_lstmunits    = [50,dim_emb],
        rho            = 0.3,
        n_iter         = 2,
        xeta           = 1e-3,
        n_batch        = 10,
        modelfile      = ['./intermediate/enc_model_dynAERNN.json', 
                            './intermediate/dec_model_dynAERNN.json'],
        weightfile     = ['./intermediate/enc_weights_dynAERNN.hdf5', 
                            './intermediate/dec_weights_dynAERNN.hdf5'],
        savefilesuffix = "testing")
    embs = []

    graphs     = [nx.Graph(data.adjs[l,:,:]) for l in range(length)]
    for temp_var in range(lookup, length):
                    emb, _ = embedding.learn_embeddings(graphs[:temp_var])
                    embs.append(emb)
    centroid=kmeans(embs[-1],data.n_class)[0] #change kSigvec from complex64 to float
    result=vq(embs[-1],centroid)[0]



    perm = permutations(range(data.n_class)) 
    one_hot_result=one_hot(result,data.n_class)
    acc_test=0
    f1_test=0
    auc_test=0
    count=0
    for i in perm: 
        count+=1
        one_hot_i=one_hot(np.array(i))
        perm_result=np.matmul(one_hot_result,one_hot_i)
        labels = np.argmax(data.labels,axis=1)
        pred_labels=np.argmax(perm_result,axis=1)
        acc_test = max(metrics.accuracy_score(labels,pred_labels),acc_test)
        f1_test=max(metrics.f1_score(labels, pred_labels,average='weighted'),f1_test)
        auc_test=max(metrics.roc_auc_score(one_hot(labels), perm_result,multi_class='ovr',average='weighted'),auc_test)
        if count%10000==0:
            print(count)
            print(acc_test,f1_test,auc_test)   
    print(str(acc_test)+'\t'+str(f1_test)+'\t'+str(auc_test))  
    try:
        spec_norm=getKlargestSigVec(adj-Probability_matrix,2)[0]
    except:
        spec_norm=[]
    return 0,acc_test,spec_norm

In [5]:
def spectral(data):
    adj = np.add.reduce(data.adjs_timestep, axis=0, keepdims=False, dtype=np.float32)

    #normalize the adj matrix
    adj += np.eye(adj.shape[0], dtype=np.float32)
    d = np.add.reduce(adj, axis=1)
    normalizing_matrix = np.zeros((adj.shape[0], adj.shape[0]))
    normalizing_matrix[range(len(normalizing_matrix)), range(len(normalizing_matrix))] = d**(-0.5)
    adj = np.matmul(normalizing_matrix,adj)
    adj=np.matmul(np.matmul(normalizing_matrix,adj), normalizing_matrix)

    Lbar=np.array(adj)  #no normalizaton
    top_k=data.n_class
    kSigVal,kSigVec=getKlargestSigVec(Lbar,top_k)
    centroid=kmeans(kSigVec.astype(float),data.n_class)[0] #change kSigvec from complex64 to float
    result=vq(kSigVec.astype(float),centroid)[0]

    
    perm = permutations(range(data.n_class)) 
    one_hot_result=one_hot(result,data.n_class)
    acc_test=0
    f1_test=0
    auc_test=0
    count=0
    for i in perm: 
        count+=1
        one_hot_i=one_hot(np.array(i))
        perm_result=np.matmul(one_hot_result,one_hot_i)
        labels = np.argmax(data.labels,axis=1)
        pred_labels=np.argmax(perm_result,axis=1)
        acc_test = max(metrics.accuracy_score(labels,pred_labels),acc_test)
        f1_test=max(metrics.f1_score(labels, pred_labels,average='weighted'),f1_test)
        auc_test=max(metrics.roc_auc_score(one_hot(labels), perm_result,multi_class='ovr',average='weighted'),auc_test)
        if count%10000==0:
            print(count)
            print(acc_test,f1_test,auc_test)   
    print(str(acc_test)+'\t'+str(f1_test)+'\t'+str(auc_test))  
    try:
        spec_norm=getKlargestSigVec(adj-Probability_matrix,2)[0]
    except:
        spec_norm=[]
    return 0,acc_test,spec_norm

In [6]:
def getKlargestSigVec(Lbar,k):
	"""input
	"matrix Lbar and k
	"return
	"k largest singular values and their corresponding eigen vectors
	"""
	lsigvec,sigval,rsigvec=linalg.svd(Lbar)
	dim=len(sigval)
 
	#find top k largest left sigval
	dictSigval=dict(zip(sigval,range(0,dim)))
	kSig=np.sort(sigval)[::-1][:k]#[0:k]
	ix=[dictSigval[k] for k in kSig]
	return sigval[ix],lsigvec[:,ix]

In [7]:
def one_hot(l,classnum=1): #classnum fix some special case
    one_hot_l=np.zeros((len(l),max(l.max()+1,classnum)))
    for i in range(len(l)):
        one_hot_l[i][l[i]]=1
    return one_hot_l

In [8]:
for n in range(0,5):
    spectral(read_data("Brain"))

0.3200354609929078	0.15898827304313726	0.5013252489334223
0.3200354609929078	0.15898827304313726	0.5013252489334223
0.3200354609929078	0.15898827304313726	0.5013252489334223
0.32092198581560283	0.16074587507313426	0.5019779904477827
0.3200354609929078	0.15898827304313726	0.5013252489334223


In [9]:
# dynaernn(read_data("Brain"))